In [ ]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.4f' % x)

from dotenv import load_dotenv
load_dotenv()
import os


import numpy as np
np.set_printoptions(suppress=True)

import ccxt
import datetime
from datetime import date
import mibian

from pandas.io.json import json_normalize

import scipy.interpolate as interpolate
from scipy.interpolate import interp1d

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
pd.options.plotting.backend = "plotly"

import warnings
warnings.filterwarnings('ignore')


deribit = ccxt.deribit()
deribit.apiKey = os.getenv('DERIBIT_KEY')
#deribit.apiKey = 'DERIBIT_KEY'
deribit.secret = os.getenv('DERIBIT_SECRET')
#deribit.secret = 'DERIBIT_SECRET'

##GVOL API ENDPOINTS 
from gvol import GVol
GVOL_API_KEY = os.getenv('GVOL_API_KEY') #VARIABLE CONTAINED IN .ENV FILE
#GVOL_API_KEY = 'GVOL_API_KEY'
gvol_client = GVol(header='x-oracle',gvol_api_key=GVOL_API_KEY)

###SELECT CURRENCY BTC/ETH
symbol ='BTC'

##OPTIONS CHAIN AND CREATING INSTRUMENTS DF FOR AUTOCOMPLETATION
instruments = pd.json_normalize(deribit.public_get_get_instruments({'currency':symbol,'kind':'option'})['result'])
instruments = instruments.sort_values(['expiration_timestamp','strike'])
instruments = pd.DataFrame(instruments['instrument_name'],columns=instruments['instrument_name'])

In [ ]:
### RETRIEVE INFORMATION OF ACTUAL PORTFOLIO FOR CURRENCY. COMMENT FOLLOW ROWS IF DONT HAVE POSITIONS

#portfolio = pd.json_normalize(deribit.private_get_get_positions({'currency':symbol})['result'])
#book = portfolio[portfolio['kind']=='option'][['instrument_name','size']]
#deltas_book = portfolio[portfolio['kind']=='future']['delta'].astype(float).sum()

#simulation = []
#for inst, size in zip(book['instrument_name'],book['size']):
#    simulation.append({'instrument':inst,'size':float(size)})
    

#print ('Deltas of book: ' +str(deltas_book))
#simulation

In [ ]:
### CREATE YOUR SIMULATION HERE 
deltas_book = 0
simulation = [
             {'instrument': 'BTC-30SEP22-20000-C', 'size': 250},
             {'instrument': 'BTC-30SEP22-25000-C', 'size': -500},
             {'instrument': 'BTC-30SEP22-30000-C', 'size': 250},         
             ]

In [ ]:
%%time
data = gvol_client.PortfolioAnalyzer(portfolio=simulation,deltaFutures=deltas_book)

book =pd.json_normalize(data['PortfolioAnalyzer'])
df = book
df['index'] = df['index'].astype(int)
#df.set_index('indexChange',inplace=True)

In [ ]:
## Insert manually your equity
equity = 0
df['equity'] = (df['PnL'] + equity).round(4)
df['equityUSD'] = (df['equity'] * df['index']).astype(int)

In [ ]:
col = ['PnL','PnLUSD','deltaBSM','deltaCash','deltaSkew','gamma','vega','wVega','theta','equity','equityUSD','days']


print ('')
print ('PORTFOLIO ANALYZER TABLE')
display(df[df['days']==0].iloc[50:115:5].set_index('index')[col]\
.style.format({
            'PnL':"{:20,.4f}",
             'PnLUSD':"${:20,.0f}",
             'deltaBSM':"{:20,.2f}",
               'deltaCash':"{:20,.2f}",
               'deltaSkew':"{:20,.2f}",
             'gamma':"{:20.4f}",
              'vega':"{:20.0f}",
              'wVega':"{:20,.0f}",
            'theta':"{:20,.0f}",
            'index':"${:20,.0f}",
            'equity':"{:20,.4f}",
            'equityUSD':"${:20,.0f}",
            'days':"{:20,.0f}"

}))

In [ ]:
%run Portfolio_analyzer_clients_charts.ipynb